Retention Redshift Insert (API Method)
* StelllarAlgo Data Science
* Ryan Kazmerik & Nakisa Rad
* Mar 7, 2022

In [1]:
import boto3
import getpass
import pandas as pd
import awswrangler as wr
import pyodbc

from datetime import datetime
from pytz import timezone


### Let's connect to MSSQL and run a stored proc to get our dataset:

In [2]:
# connect to SQL Server.
SERVER = '52.44.171.130' 
DATABASE = 'datascience' 
USERNAME = 'dsAdminWrite' 
PASSWORD = getpass.getpass(prompt='Enter your password')
CNXN = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+USERNAME+';PWD='+ PASSWORD)

Enter your password ····················


In [3]:

cursor = CNXN.cursor()

storedProc = (
    f"""select * from ds.productyear_all"""
)

df = pd.read_sql(storedProc, CNXN)

CNXN.commit()
cursor.close()

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113 entries, 0 to 112
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   teamproductyearid  113 non-null    int64  
 1   lkupclientid       113 non-null    int64  
 2   clientcode         113 non-null    object 
 3   productgrouping    113 non-null    object 
 4   trainseasonyear    113 non-null    int64  
 5   testseasonyear     113 non-null    int64  
 6   isprevyearflag     113 non-null    bool   
 7   facttestyear       113 non-null    int64  
 8   facttestprevyear   65 non-null     float64
 9   stlrDBName         113 non-null    object 
 10  leagueName         113 non-null    object 
dtypes: bool(1), float64(1), int64(5), object(4)
memory usage: 9.1+ KB


### First we have to tell AWS which profile we'd like to login to, this will send us for a browser authentication trip:

In [4]:
! aws sso login --profile Stellaralgo-DataScienceAdmin

Attempting to automatically open the SSO authorization page in your default browser.
If the browser does not open or you wish to use a different device to authorize this request, open the following URL:

https://device.sso.us-east-1.amazonaws.com/

Then enter the code:

JKLC-LFFV
Successully logged into Start URL: https://stellaralgo.awsapps.com/start#/


### Now we can create a session and client to RedShift, and create a new connection using AWS wrangler:

In [5]:
try:
    session = boto3.setup_default_session(profile_name='Stellaralgo-DataScienceAdmin')
    client = boto3.client('redshift')

    conn = wr.data_api.redshift.connect(
        cluster_id="qa-app",
        database="datascience",
        db_user="admin"
    )
    
    print('CREDENTIALS RETRIEVED SUCCESSFULLY')
    
except Exception as err:
    print('FAILED TO RETRIEVE CREDENTIALS!', err)

CREDENTIALS RETRIEVED SUCCESSFULLY


### Insert the new scores into the customerRetentionScores table:

In [14]:
try:
    fields = f"""
    INSERT INTO datascience.ds.teamsconfig (
        teamproductyearid,
        lkupclientid,
        clientcode,
        productgrouping,
        trainseasonyear,
        testseasonyear,
        isprevyearflag,
        stlrdbname,
        leaguename
    ) VALUES """

    # Insert Dataframe into SQL Server:
    values_list = []
    for i, (index, row) in enumerate(df.iterrows()):

        values = f"""
            (
                {str(row.teamproductyearid)},
                {str(row.lkupclientid)},
                '{str(row.clientcode)}',
                '{str(row.productgrouping)}',
                {str(row.trainseasonyear)},
                {str(row.testseasonyear)},
                {str(row.isprevyearflag)},
                '{str(row.stlrDBName)}',
                '{str(row.leagueName)}'
            )"""

        values_list.append(values)
        
        insert_statement = fields + ",".join(values_list)+";"

        wr.data_api.redshift.read_sql_query(
            sql = insert_statement, 
            con = conn
        )

except Exception as err:
    print("ERROR WHILE INSERTING TEAMS CONFIG TO DATABASE:", err)

print("TEAMS CONFIG INSERTS COMPLETED")

RETENTION SCORE INSERTS COMPLETED
